In [1]:
import os, sys
from datasets import *
import numpy as np
import pandas as pd

homedir = "../"
datadir = "../data/facescrub/"
sys.path.append(datadir)

In [2]:
actors_file = os.path.join(datadir, "new_subset_actors.txt")
actresses_file = os.path.join(datadir, "new_subset_actresses.txt")
target_folder = os.path.join(datadir, "uncropped/")

In [3]:
# Some constants
img_size = 28
input_size = img_size * img_size * 3
trainsize = 70
ready_data = os.path.join(datadir, "data_size_28_training_70.parquet")


# Construct master dataset
master_dataset = FaceScrubDataset(
    actors_file=actors_file, actresses_file=actresses_file, target_image_size=img_size, 
    target_folder=target_folder, transform=None, train_size=trainsize, get_data_at_first_call=False,
    delete_image_after_download=True, verbose=False)
output_size = len(master_dataset.artists_list)
master_dataset.load_data(ready_data)

/home/pouya/anaconda3/envs/comp541/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: URLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  if sys.path[0] == "":


In [6]:
# Constructing data with 224 pixels rather than 28.
# Dataset object is the same. We will only change its target_image_size property.
# We do not need to download and save all the images, because our dataset is alrewady processed, 
# and we know which URLs are available to use. We can more easily use direct indexing in pytorch dataset.

# Setting some parameters to avoid direct indexing from arrays in the datasset
master_dataset.target_image_size = 224
master_dataset.data_is_processed = False
master_dataset.get_data_at_first_call = False
master_dataset.images = []
master_dataset.labels = []
master_dataset.df = master_dataset.df.iloc[:,:13]
# We will not touch the 'labels' property because that one will not change.
# We will not touch the 'df' property because that one will not change.

# Since we will use live indexing, we will have to use num_workers=0 
# to avoid data race and other multiprocessing issues.

In [8]:
master_dataset.process_all_data()
print(master_dataset.images.shape)
print(master_dataset.labels.shape)
print(master_dataset.df.shape)

Progress: 100%|█████████████████████████████████████████████████| 2650/2650 [46:54<00:00,  1.06s/it]

(4963, 224, 224, 3)
(4963,)
(4963, 13)


In [13]:
np.savez(make_path(os.path.join(datadir, "data_size_224_training_70", "data.npz")), 
         images=master_dataset.images, labels=master_dataset.labels, 
         artists=np.array(master_dataset.artists_list, dtype=str))
master_dataset.df.to_parquet(make_path(os.path.join(datadir, "data_size_224_training_70", "info.parquet")))

/home/pouya/anaconda3/envs/comp541/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.str` is a deprecated alias for the builtin `str`. To silence this warning, use `str` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.str_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
